In [ ]:
"""Shared inference utilities extracted from the original notebook.

This module centralizes the ML, CV, and optional RL inference steps that were
previously defined inside ``app/inference_pipeline.ipynb`` so that they can be
imported and reused without executing notebook cells.
"""
from __future__ import annotations

import json
import os
from datetime import timedelta
from functools import lru_cache
from typing import Any, Dict, Optional, Tuple

import numpy as np
import pandas as pd
import pytz
from dotenv import load_dotenv
from tinkoff.invest import CandleInterval, Client

try:  # ML
    import xgboost as xgb
except ImportError:  # pragma: no cover - optional dependency
    xgb = None  # type: ignore

try:  # CV
    import torch
    from torch import nn
    from torchvision import transforms
except ImportError:  # pragma: no cover - optional dependency
    torch = None  # type: ignore
    nn = None  # type: ignore
    transforms = None  # type: ignore

try:  # CV backbone
    import timm
except ImportError:  # pragma: no cover - optional dependency
    timm = None  # type: ignore

try:  # Candle rendering
    import mplfinance as mpf
    import matplotlib.pyplot as plt
except ImportError:  # pragma: no cover - optional dependency
    mpf = None  # type: ignore
    plt = None  # type: ignore

from models.RL import build_feature_frame
from models.RL.agent import RLAgent
from models.RL.env import PortfolioState

ModuleNotFoundError: No module named 'models'

In [4]:

load_dotenv()

DEFAULT_TOKEN = os.getenv("TINKOFF_TOKEN")
DEFAULT_TZ = pytz.timezone("Europe/Moscow")
DEFAULT_TIMEFRAME = CandleInterval.CANDLE_INTERVAL_5_MIN

In [5]:
def fetch_tinkoff_candles(
    token: str,
    ticker: str,
    days: int = 1,
    interval: CandleInterval = DEFAULT_TIMEFRAME,
    tz: pytz.BaseTzInfo = DEFAULT_TZ,
) -> pd.DataFrame:
    """Download OHLCV candles from the Tinkoff Invest API."""

    if not token:
        raise RuntimeError("TINKOFF_TOKEN is required to fetch candles")

    from tinkoff.invest.services import InstrumentsService  # lazy import
    from tinkoff.invest.utils import now

    with Client(token) as client:
        instruments: InstrumentsService = client.instruments
        shares = instruments.shares().instruments
        figi: Optional[str] = None
        for share in shares:
            if share.ticker.upper() == ticker.upper():
                figi = share.figi
                break
        if not figi:
            raise RuntimeError(f"FIGI for {ticker} not found")

        end = now()
        start = end - timedelta(days=days)
        candles = client.get_all_candles(figi=figi, from_=start, to=end, interval=interval)

        data = [
            {
                "time": candle.time.astimezone(tz),
                "open": candle.open.units + candle.open.nano / 1e9,
                "high": candle.high.units + candle.high.nano / 1e9,
                "low": candle.low.units + candle.low.nano / 1e9,
                "close": candle.close.units + candle.close.nano / 1e9,
                "volume": candle.volume,
            }
            for candle in candles
        ]

    return pd.DataFrame(data)


In [6]:

def compute_ml_features(df: pd.DataFrame) -> pd.DataFrame:
    """Compute the feature set used by the XGBoost model."""

    df = df.copy().reset_index(drop=True)
    df["close"] = df["close"].astype(float)
    df["logret"] = np.log(df["close"]).diff()
    df["ret_1"] = df["logret"].shift(1)
    for w in [3, 5, 10]:
        df[f"sma_{w}"] = df["close"].rolling(window=w).mean()
        df[f"std_{w}"] = df["logret"].rolling(window=w).std()
    df["high"] = df["high"].astype(float)
    df["low"] = df["low"].astype(float)
    tr1 = df["high"] - df["low"]
    tr2 = (df["high"] - df["close"].shift(1)).abs()
    tr3 = (df["low"] - df["close"].shift(1)).abs()
    df["tr"] = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    df["atr_14"] = df["tr"].rolling(14).mean()
    feats = ["ret_1", "sma_3", "sma_5", "sma_10", "std_3", "std_5", "std_10", "atr_14"]
    return df[feats]


def load_xgb_model(model_path: str) -> "xgb.Booster":
    """Load a saved XGBoost model from file."""

    if xgb is None:
        raise ImportError(
            "xgboost is not installed. Install it via `pip install xgboost` to use the ML inference pipeline."
        )
    booster = xgb.Booster()
    booster.load_model(model_path)
    return booster


def predict_ml(model: "xgb.Booster", feats_df: pd.DataFrame, return_proba: bool = True) -> Dict[str, Any]:
    """Run ML model inference on feature DataFrame."""

    dmatrix = xgb.DMatrix(feats_df)
    probs = model.predict(dmatrix)
    preds = np.argmax(probs, axis=1)
    idx_to_label = {0: "down", 1: "flat", 2: "up"}
    labels = [idx_to_label.get(int(i), str(i)) for i in preds]
    result: Dict[str, Any] = {"indices": preds.tolist(), "labels": labels}
    if return_proba:
        result["proba"] = probs
    return result


In [7]:
def add_cv_features(df: pd.DataFrame) -> pd.DataFrame:
    """Compute EMA and Bollinger band overlays for candlestick rendering."""

    df = df.copy()
    df["ema_10"] = df["close"].ewm(span=10).mean()
    df["ema_20"] = df["close"].ewm(span=20).mean()
    mid = df["close"].rolling(20).mean()
    std = df["close"].rolling(20).std()
    df["boll_up"] = mid + 2 * std
    df["boll_low"] = mid - 2 * std
    return df


def render_candle_image(
    sub_df: pd.DataFrame,
    img_size: tuple = (8, 4),
    dpi: int = 100,
    use_jpg: bool = True,
    style: Optional[Any] = None,
) -> np.ndarray:
    """Render a single candlestick window into a NumPy RGB image."""

    if mpf is None or plt is None:
        raise ImportError(
            "mplfinance and matplotlib must be installed to render candle images; install them via `pip install mplfinance matplotlib`"
        )

    if style is None:
        mc = mpf.make_marketcolors(up="lime", down="red", edge="white", wick="white", volume="gray")
        style = mpf.make_mpf_style(
            base_mpf_style="nightclouds",
            facecolor="black",
            edgecolor="white",
            marketcolors=mc,
            rc={"axes.labelcolor": "white", "axes.edgecolor": "white"},
        )

    fig, axes = mpf.plot(
        sub_df,
        type="candle",
        style=style,
        volume=True,
        figsize=img_size,
        tight_layout=True,
        show_nontrading=True,
        returnfig=True,
    )
    for ax in axes:
        ax.set_axis_off()
        ax.grid(False)
    if "ema_10" in sub_df.columns:
        axes[0].plot(sub_df.index, sub_df["ema_10"], color="deepskyblue", linewidth=1)
    if "ema_20" in sub_df.columns:
        axes[0].plot(sub_df.index, sub_df["ema_20"], color="orange", linewidth=1)
    if {"boll_up", "boll_low"}.issubset(sub_df.columns):
        axes[0].plot(sub_df.index, sub_df["boll_up"], color="gray", linestyle="--", linewidth=0.8)
        axes[0].plot(sub_df.index, sub_df["boll_low"], color="gray", linestyle="--", linewidth=0.8)

    fig.canvas.draw()
    buf = fig.canvas.buffer_rgba()
    w, h = fig.canvas.get_width_height()
    img = np.frombuffer(buf, dtype=np.uint8).reshape((h, w, 4))[..., :3]
    plt.close(fig)
    return img


def prepare_cv_tensor(img: np.ndarray) -> "torch.Tensor":
    """Convert an RGB image array into a Torch tensor with normalization."""

    if torch is None or transforms is None:
        raise ImportError("torch and torchvision must be installed to prepare image tensors")

    transform = transforms.Compose(
        [
            transforms.ToPILImage(),
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ]
    )
    return transform(img).unsqueeze(0)


def load_cv_model(meta_path: str, model_path: str, device: Optional[str] = None) -> "nn.Module":
    """Reconstruct and load the CV model from saved state."""

    if timm is None or torch is None or nn is None:
        raise ImportError("timm and torch must be installed to load the CV model")

    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    with open(meta_path, "r") as f:
        meta = json.load(f)
    label_to_idx: Dict[str, int] = meta["label_to_idx"]
    model_name: str = meta.get("model_name", "convnext_tiny")
    num_classes = len(label_to_idx)

    backbone = timm.create_model(model_name, pretrained=False, num_classes=0, global_pool="avg")
    feat_dim = backbone.num_features  # type: ignore[attr-defined]
    head = nn.Sequential(
        nn.Linear(feat_dim, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes),
    )
    model = nn.Sequential(backbone, head)

    checkpoint = torch.load(model_path, map_location=device)
    state_dict = checkpoint["model_state"] if isinstance(checkpoint, dict) and "model_state" in checkpoint else checkpoint
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    return model


def predict_cv(
    model: "nn.Module",
    img_tensor: "torch.Tensor",
    meta_path: str,
    device: Optional[str] = None,
    return_proba: bool = True,
) -> Dict[str, Any]:
    """Run the CV model on a single image tensor."""

    if torch is None:
        raise ImportError("torch must be installed to perform CV inference")

    device = device or ("cuda" if torch.cuda.is_available() else "cpu")
    img_tensor = img_tensor.to(device)
    with torch.no_grad():
        logits = model(img_tensor)
        probs = torch.softmax(logits, dim=1)
        pred_idx = probs.argmax(dim=1).cpu().numpy()
        probs_np = probs.cpu().numpy()

    with open(meta_path, "r") as f:
        meta = json.load(f)
    idx_to_label = {v: k for k, v in meta["label_to_idx"].items()}
    labels = [idx_to_label.get(int(i), str(i)) for i in pred_idx]

    result: Dict[str, Any] = {"indices": pred_idx.tolist(), "labels": labels}
    if return_proba:
        result["proba"] = probs_np
    return result



In [8]:
@lru_cache(maxsize=1)
def load_rl_agent(model_path: str, window_size: int, initial_balance: float) -> RLAgent:
    """Load and cache the PPO-based RL agent for inference."""

    return RLAgent(model_path=model_path, window_size=window_size, initial_balance=initial_balance)


def prepare_rl_state(
    df: pd.DataFrame,
    window_size: int,
    initial_balance: float,
    position: int = 0,
    equity: Optional[float] = None,
) -> Tuple[pd.DataFrame, PortfolioState]:
    """Compute RL feature frame and latest portfolio state for action selection."""

    features = build_feature_frame(df)
    if features.empty:
        raise ValueError("Cannot build RL features from an empty DataFrame")

    pointer = len(features) - 1
    state = PortfolioState(
        pointer=pointer,
        position=position,
        equity=equity if equity is not None else initial_balance,
    )
    return features, state

In [9]:
def run_full_inference(
    ticker: str = "SBER",
    tf: CandleInterval = DEFAULT_TIMEFRAME,
    days: int = 1,
    lookback: int = 60,
    ml_model_path: str = "xgb_sber.model",
    cv_model_path: str = "best_model.pth",
    meta_path: str = "meta.json",
    token: Optional[str] = DEFAULT_TOKEN,
    tz_name: str = "Europe/Amsterdam",
    use_rl: bool = False,
    rl_model_path: str = "models/RL/checkpoints/ppo_trading_agent.zip",
    rl_window: int = 32,
    initial_balance: float = 100_000.0,
    portfolio_position: int = 0,
    portfolio_equity: Optional[float] = None,
) -> Dict[str, Any]:
    """Run ML, CV, and optional RL models on the most recent candle data."""

    tz = pytz.timezone(tz_name)
    df_candles = fetch_tinkoff_candles(token or "", ticker, days=days, interval=tf, tz=tz)
    if df_candles.empty:
        raise RuntimeError("No candle data available for inference")
    df_candles = df_candles.sort_values("time").reset_index(drop=True)

    ml_model = load_xgb_model(ml_model_path)
    feats_df = compute_ml_features(df_candles).dropna()
    ml_result = predict_ml(ml_model, feats_df, return_proba=True)

    df_feat = add_cv_features(df_candles)
    if len(df_feat) < lookback:
        raise ValueError(
            f"Not enough candles ({len(df_feat)}) for lookback={lookback}. Increase history length or decrease lookback."
        )
    sub = df_feat.iloc[-lookback:].copy()
    sub["time"] = pd.to_datetime(sub["time"])
    sub = sub.set_index("time")
    img = render_candle_image(sub, img_size=(8, 4), dpi=100, use_jpg=True)
    img_tensor = prepare_cv_tensor(img)
    cv_model = load_cv_model(meta_path, cv_model_path)
    cv_result = predict_cv(cv_model, img_tensor, meta_path, return_proba=True)

    rl_action = None
    rl_action_label = None
    rl_state = None
    rl_features = None
    if use_rl:
        agent = load_rl_agent(rl_model_path, rl_window, initial_balance)
        rl_features, rl_state = prepare_rl_state(
            df_candles,
            window_size=rl_window,
            initial_balance=initial_balance,
            position=portfolio_position,
            equity=portfolio_equity if portfolio_equity is not None else initial_balance,
        )
        rl_action, rl_action_label = agent.get_action(rl_features, rl_state)

    result: Dict[str, Any] = {"ml_preds": ml_result, "cv_preds": cv_result, "df_candles": df_candles}
    if use_rl:
        result.update(
            {
                "rl_action": rl_action,
                "rl_action_label": rl_action_label,
                "rl_portfolio_state": rl_state,
                "rl_features": rl_features.tail(rl_window) if rl_features is not None else None,
            }
        )

    return result
